In [15]:
import pandas as pd
from logs_parser.parser import parse_mjlog
from logs_parser.viewer import print_node
import xml.etree.ElementTree as ET
import preprocess
import sys
import json

df = pd.read_csv('./logs_parser/2021.csv')
for i, xml_str in enumerate(df['log_content']):
    node = ET.fromstring(xml_str)
    data = parse_mjlog(node)
    if len(data["meta"]['UN'][3]["name"]) == 0:
        continue

    if i == 2:
        f = open('example.json','w')
        f.write(json.dumps(data, indent=2))
        f.close()
        sys.exit()
    res = {}
    for i in range(len(data["rounds"])):

        round_data = data["rounds"][i] 
        round_info = preprocess.get_round_info(round_data) 


SystemExit: 

In [16]:
from mahjong.hand_calculating.hand import HandCalculator
from mahjong.tile import TilesConverter
from mahjong.hand_calculating.hand_config import HandConfig
from mahjong.meld import Meld
from mahjong.shanten import Shanten


def getFeatures(round_info):
    features = {}
    

    def shanten(tiles_list):
        #given 14
        tiles = TilesConverter.to_34_array(tiles_list)
        return Shanten.calculate_shanten(tiles)

    def doraAround(round_info):
        dora_indicators = np.array(round_info['dora_indicators']) // 4
        res = 0
        for t in tiles:
            t //= 4
            for d in dora_indicators:
                if abs(d-t) <= 2:
                    res += 1
                    break
        return res
    

In [56]:
from typing import List
from utils import *
shanten_calculator = Shanten()

def is_tile_strictly_isolated(hand_34, tile_34):
    """
    Tile is strictly isolated if it doesn't have -2, -1, 0, +1, +2 neighbors
    :param hand_34: array of tiles in 34 tile format
    :param tile_34: int
    :return: bool
    """

    if is_honor(tile_34):
        return hand_34[tile_34] - 1 <= 0

    simplified = simplify(tile_34)

    # 1 suit tile
    if simplified == 0:
        indices = [tile_34, tile_34 + 1, tile_34 + 2]
    # 2 suit tile
    elif simplified == 1:
        indices = [tile_34 - 1, tile_34, tile_34 + 1, tile_34 + 2]
    # 8 suit tile
    elif simplified == 7:
        indices = [tile_34 - 2, tile_34 - 1, tile_34, tile_34 + 1]
    # 9 suit tile
    elif simplified == 8:
        indices = [tile_34 - 2, tile_34 - 1, tile_34]
    # 3-7 tiles tiles
    else:
        indices = [tile_34 - 2, tile_34 - 1, tile_34, tile_34 + 1, tile_34 + 2]

    isolated = True
    for tile_index in indices:
        # we don't want to count our tile as it is in hand already
        if tile_index == tile_34:
            isolated &= hand_34[tile_index] - 1 <= 0
        else:
            isolated &= hand_34[tile_index] == 0

    return isolated



def calculate_waits(closed_hand_34: List[int], all_tiles_34: List[int], use_chiitoitsu: bool = True):
    shanten = Shanten()
    previous_shanten = shanten.calculate_shanten(closed_hand_34, use_chiitoitsu=use_chiitoitsu)

    waiting = []
    for tile_index in range(0, 34):
        # it is important to check that we don't have all 4 tiles in the all tiles
        # not in the closed hand
        # so, we will not count 1z as waiting when we have 111z meld
        if all_tiles_34[tile_index] == 4:
            continue

        closed_hand_34[tile_index] += 1

        skip_isolated_tile = True
        if closed_hand_34[tile_index] == 4:
            skip_isolated_tile = False
        if use_chiitoitsu and closed_hand_34[tile_index] == 3:
            skip_isolated_tile = False

        # there is no need to check single isolated tile
        if skip_isolated_tile and is_tile_strictly_isolated(closed_hand_34, tile_index):
            closed_hand_34[tile_index] -= 1
            continue

        new_shanten = shanten.calculate_shanten(closed_hand_34, use_chiitoitsu=use_chiitoitsu)

        if new_shanten == previous_shanten - 1:
            waiting.append(tile_index)
#             print('isolated:')
#             print("{0:27b}".format(shanten.number_isolated_tiles))

        closed_hand_34[tile_index] -= 1

    return waiting, previous_shanten


def getUkeire(closed_hand_34, all_tiles_34):
    '''
    given hand with 13 tiles
    '''
    return calculate_waits(closed_hand, all_tiles_34)

# def shanten1(closed_hand_34, all_tiles_34):
#     ukeire = getUkeire(closed_hand34, all_tiles_34)
#     for t in ukeire:


closed_hand = TilesConverter.string_to_34_array(man='12334556889', sou='13')
# # print(closed_hand)
shanten = Shanten()
print(shanten.calculate_shanten(closed_hand))
print("{0:27b}".format(shanten.number_isolated_tiles))
print(shanten.number_jidahai)
# calculate_waits(closed_hand, closed_hand)

1
      101000000000110110000
0


In [80]:
def head1(closed_hand_34, all_tiles_34):
    '''
    try to update hands one step further
    '''
    ukeire, shanten = calculate_waits(closed_hand_34, all_tiles_34)
    print(ukeire, shanten)
    
    res = []
    for u in ukeire:
        closed_hand_34[u] += 1
        all_tiles_34[u] += 1
            
        for i in range(34):    #random discard a close tile to see if it is isolated
            if closed_hand_34[i] <= 0:
                continue
            all_tiles_34_copy = all_tiles_34[:]
            closed_hand_34_copy = closed_hand_34[:]
            all_tiles_34_copy[i] -= 1
            closed_hand_34_copy[i] -= 1
            new_ukeire, new_shanten = calculate_waits(closed_hand_34_copy, all_tiles_34_copy)
            if new_shanten < shanten:
                res.append((u, i))
        closed_hand_34[u] -= 1
        all_tiles_34[u] -= 1
            
    return res
closed_hand = TilesConverter.string_to_34_array(man='34588', sou='4478', pin='2379')


head1(closed_hand[:], closed_hand[:])

[7, 9, 12, 16, 21, 23, 26] 2


[(7, 10),
 (7, 11),
 (7, 15),
 (7, 17),
 (7, 24),
 (7, 25),
 (9, 7),
 (9, 15),
 (9, 17),
 (9, 21),
 (9, 24),
 (9, 25),
 (12, 7),
 (12, 15),
 (12, 17),
 (12, 21),
 (12, 24),
 (12, 25),
 (16, 7),
 (16, 10),
 (16, 11),
 (16, 21),
 (16, 24),
 (16, 25),
 (21, 10),
 (21, 11),
 (21, 15),
 (21, 17),
 (21, 24),
 (21, 25),
 (23, 7),
 (23, 10),
 (23, 11),
 (23, 15),
 (23, 17),
 (23, 21),
 (26, 7),
 (26, 10),
 (26, 11),
 (26, 15),
 (26, 17),
 (26, 21)]

In [140]:
from mahjong.hand_calculating.hand import HandCalculator
from collections import defaultdict
def search(closed_hand_34, all_tiles_34):
    '''
    for now just process with shanten within 3
    '''
    
    hand_calculator = HandCalculator()

    
    q = []
    res = defaultdict(int)
    visited = set()
    q.append((closed_hand_34, all_tiles_34))
    while q:
        ch, at = q[0]
        q.pop(0)
        visited.add(TilesConverter.to_one_line_string(TilesConverter.to_136_array(at)))
        ukeire, shanten = calculate_waits(ch, at) 
     
        for u in ukeire:
            ch[u] += 1
            at[u] += 1
            shanten_after_draw = shanten_calculator.calculate_shanten(at)
            if shanten_after_draw == -1:
                wintile = [0] * 34
                wintile[u] = 1
                value = hand_calculator.estimate_hand_value(
                    TilesConverter.to_136_array(at), 
                    TilesConverter.to_136_array(wintile)[0]
                    )
                res[TilesConverter.to_one_line_string(TilesConverter.to_136_array(at))] += 1
            else:
                for i in range(34):    #random discard a close tile to see if it is isolated
                    if ch[i] <= 0:
                        continue
                    all_tiles_34_copy = at[:]
                    closed_hand_34_copy = ch[:]
                    all_tiles_34_copy[i] -= 1
                    closed_hand_34_copy[i] -= 1
                    if TilesConverter.to_one_line_string(TilesConverter.to_136_array(all_tiles_34_copy)) in visited:
                        continue
                    new_ukeire, new_shanten = calculate_waits(closed_hand_34_copy, all_tiles_34_copy)
                    if new_shanten < shanten:
                        if sum(all_tiles_34_copy) != 13:
                            import ipdb; ipdb.set_trace()
                        q.append((closed_hand_34_copy, all_tiles_34_copy))
            ch[u] -= 1
            at[u] -= 1 
    return res
# closed_hand = TilesConverter.one_line_string_to_34_array('34m24689p37889s3z')
closed_hand = TilesConverter.string_to_34_array(man='34588', sou='4478', pin='2379')
# closed_hand = TilesConverter.string_to_34_array(man='1112345678999')
possible_ends = search(closed_hand[:], closed_hand[:])
print(possible_ends)

defaultdict(<class 'int'>, {'345888m789p44678s': 12, '345888m789p44789s': 12, '345888m123p44678s': 12, '345888m123p44789s': 12, '345888m234p44678s': 12, '345888m234p44789s': 12, '345888m123789p44s': 12, '345888m234789p44s': 12, '345m123789p44678s': 6, '345m123789p44789s': 6, '34588m123p444678s': 12, '34588m123p444789s': 12, '34588m123789p678s': 6, '34588m123789p789s': 6, '34588m123789p444s': 12, '345m234789p44678s': 6, '345m234789p44789s': 6, '34588m234p444678s': 12, '34588m234p444789s': 12, '34588m234789p678s': 6, '34588m234789p789s': 6, '34588m234789p444s': 12, '34588m789p444678s': 12, '34588m789p444789s': 12})


In [88]:
closed_hand = TilesConverter.string_to_34_array(man='11112345678999')
shanten_calculator.calculate_shanten(closed_hand[:], closed_hand[:])

-1

In [135]:
len(possible_ends)

70